In [ ]:
import ast
import unicodedata
import numpy as np
import pandas as pd

from tqdm import tqdm
from src.utils_data_wrangling import *

In [ ]:
car_features = pd.DataFrame()
failed_urls = pd.DataFrame()

for i in [j for j in range(500, 29500, 500)] + ['final']:
    aux_features = pd.read_csv(f'data/partial_data/cars_features_{i}.csv')
    aux_failed = pd.read_csv(f'data/partial_data/failed_urls_{i}.csv')

    car_features = pd.concat([car_features, aux_features], axis=0)
    failed_urls = pd.concat([failed_urls, aux_failed], axis=0)

car_features = car_features.rename(columns={'other_fatures': 'other_features'})

In [ ]:
failed_urls['error'].value_counts()

### Get possible values for feature cols

In [ ]:
car_features['maker'] = car_features['maker'].str.lower()
car_features['model'] = car_features['model'].str.lower()
car_features['other_features'] = car_features['other_features'].str.lower()
car_features['city'] = car_features['city'].str.lower()
car_features['price'] = car_features['price'].replace({
    'R\$': '',  # Remove R$
    '\s': '',   # Remove spaces
    '\.': ''    # Remove dots
}, regex=True).astype(float)
car_features['year'] = np.where(car_features['year'] == 2924, 2024, car_features['year'])
car_features['engine'] = np.where(car_features['engine'] == '1.6/s', '1.6', car_features['engine'])

# maker - ok
# model - ok
years = [str(int(i)) for i in range(1950, 2026)] + [str(float(i)) for i in range(1950, 2026)]
engines = list(
    car_features
    .query("engine not in ('Cj-5', 'Cj-3', 'Elétrico', '2p', 'B12p', 'Ev', '44,9', 'P6', '82,5', 'P8', 'Se')")
    .astype({'engine': float})
    .groupby('engine', as_index=False)
    [['id']]
    .count()
    .sort_values('engine', ascending=True)
    .query("engine < 27")
    ['engine']
) + ['Cj-5', 'Cj-3', 'Elétrico', 'Ev', 'P6', 'P8', '1.050']
engines = [str(i) for i in engines]
transmissions = ['Manual', 'Automático', 'Automático CVT', 'Automatizado', '100% Elétrico', 'Alcool', 'Elétrico']
fuel_types = ['Flex', 'Gasolina', 'Diesel', 'Alcool', 'Híbrido e Flex', 'Híbrido e Gasolina',
              'Elétrico', 'GNV e Gasolina', 'GNV e Flex', 'GNV'] 
body_types = ['Hatch', 'SUV', 'Sedan', 'Picape', 'Monovolume', 'SW-Perua', 'Van', 
              'Furgão Pequeno', 'Minivan', 'Coupé-Fast Back', 'Conversível']
colors = (
    car_features
    .groupby('color', as_index=False)
    [['id']]
    .count()
    .sort_values('id', ascending=False)
    [lambda x: ~x['color'].str.contains(r'\d+\.?\d*\s*KM', na=False)]
    ['color']
) 
colors = [i for i in colors]
# mileage - ok
# doors e valves são meio inuteis na real, o que mais tem nessas colunas são infos de modelo 
# do carro, tipo civic ls, prisma joy e afins

### Get the correct info for each feature col

In [ ]:
car_features = (
    car_features
    .pipe(get_spread_info, 'year', years)
    .apply(lambda x: x.astype(str))
    .pipe(get_spread_info, 'engine', engines)
    .pipe(get_spread_info, 'transmission', transmissions)
    .pipe(get_spread_info, 'fuel_type', fuel_types)
    .pipe(get_spread_info, 'body_type', body_types)
    .pipe(get_spread_info, 'color', colors)
    .pipe(get_spread_mileage_info)
    .astype({'mileage': str, 'year': str})
    .assign(year = lambda x: np.where(~x['year'].isin(years), np.nan, x['year']))
    .assign(engine = lambda x: np.where(~x['engine'].isin(engines), np.nan, x['engine']))
    .assign(transmission = lambda x: np.where(~x['transmission'].isin(transmissions), np.nan, x['transmission']))
    .assign(fuel_type = lambda x: np.where(~x['fuel_type'].isin(fuel_types), np.nan, x['fuel_type']))
    .assign(body_type = lambda x: np.where(~x['body_type'].isin(body_types), np.nan, x['body_type']))
    .assign(color = lambda x: np.where(~x['color'].isin(colors), np.nan, x['color']))
    .assign(mileage = lambda x: np.where(~x['mileage'].str.endswith(('KM', "KM KM")), np.nan, x['mileage']))
    .assign(year = lambda x: np.where(~x['spread_year'].isnull(), x['spread_year'], x['year']))
    .assign(engine = lambda x: np.where(~x['spread_engine'].isnull(), x['spread_engine'], x['engine']))
    .assign(transmission = lambda x: np.where(~x['spread_transmission'].isnull(), x['spread_transmission'], x['transmission']))
    .assign(fuel_type = lambda x: np.where(~x['spread_fuel_type'].isnull(), x['spread_fuel_type'], x['fuel_type']))
    .assign(body_type = lambda x: np.where(~x['spread_body_type'].isnull(), x['spread_body_type'], x['body_type']))
    .assign(color = lambda x: np.where(~x['spread_color'].isnull(), x['spread_color'], x['color']))
    .assign(mileage = lambda x: np.where(~x['spread_mileage'].isnull(), x['spread_mileage'], x['mileage']))
    .drop(['spread_year', 'spread_engine', 'spread_transmission', 'spread_fuel_type', 'spread_body_type', 'spread_color', 'spread_mileage'], axis=1)
)

## Working with other_features col

### Making them usable

In [ ]:
extra_features = []
for item in [sublista for sublista in car_features['other_features']]:
    if item != '[]':
        try:
            lista_convertida = ast.literal_eval(item)
            extra_features.extend(lista_convertida)
        except:
            continue

extra_features = [fix_string(item) for item in extra_features]
extra_features = list(set(extra_features))

In [ ]:
pd.DataFrame({'extra_features': extra_features}).to_clipboard()

### Creating a col for each extra features and populating them

In [ ]:
for feature in extra_features:
    car_features[feature] = np.nan

car_features['other_features'] = car_features.apply(lambda x: fix_items(x), axis=1)

for feature in tqdm(extra_features):
    car_features[feature] = car_features.apply(lambda x: check_feature(x, feature), axis=1)

### Fixing redundant cateogories

In [ ]:
car_features = (
    car_features
    .pipe(merge_categories, ['direcao_hidraulica', 'direcao_eletrica', 'direcao_eletro_hiraulica', 'dire&', 'direcao_eletro-hidraulica'], 'DIRECAO')
    .pipe(merge_categories, ['ar_condicionado', 'ar_quente', 'ar_condicionado_digital', 'ar-condicionado', 'ar_digital_dualzone',
                             'ar_condicionado_dual_zone', 'ar_condicionado_adaptado', 'ar_climatronic', 'ar.cond._dual_zone',
                             'ar_digital', 'ar-condicionado_digital', 'ar_condicionado_automatico', 'ar-condicionado_dual_zone',
                             'ar_digital_dual_zone', 'saida_de_ar-condicionado_para_os_bancos_traseiros', 'ar-condicional_dual_zone'], 'AR')
    .pipe(merge_categories, ['trava_eletricas', 'trava_eletrica', 'travas_eletricas', 'alarme_antifurto_perimetrico', 
                             'trava_eletrica_central', 'travas_el&', 'trava_central'], 'TRAVAS')
    .pipe(merge_categories, ['rodas_de_liga_leves', 'rodas_de_liga_leve', 'rodas_de_liga_aro_17', 'rodas', 'roda_liga_leve_17',
                             'rodas_esportivas', 'rodas_gsi_aro_17', 'rodas_liga_leve', 'rodas_liga_leve_aro_17'], 'ARO_LIGA_LEVE')
    .pipe(merge_categories, ['alarme', 'alarme_original', 'alarme_a_distancia', 'alarme_na_chave', 'alarme_anti-furto'], 'ALARME')
    .pipe(merge_categories, ['2_air_bags', '4_air_bags', '6_air_bags', 'air_bag_duplo', 'air_bag', 'airbag', '10_air_bags', 
                             'airbag_motorista', 'airbag_passageiro', 'air_bag_do_motorista', '1_air_bag', 'airbag_duplo', 
                             'airbags_frontais/laterais/cortina', 'airbag_laterais', 'air_bag_lateral', 'airbag_cortina', 
                             'airbag_de_cortina', '12_air_bags', 'air_bags', 'airbag_de_joelho', 'air_bag_passageiro',
                             'airbag_lateral_em_modo_cortina', 'air_bag_cortina', 'airbags_frontais', '9_air_bags', 'airbag_do_motorista',
                             'airbag_(so_motorista)', 'airbag_do_passageiro', 'air_bag_motorista', 'air_bag_do_passageiro',
                             '8_air_bags', 'air_bag_duplo_e_lateral', '7_air_bags', 'airbags_frontais/laterais', 'airbag_lateral',
                             'air_bag_de_cortina', 'air_bag_lateral_dianteiro'], 'AIRBAG')
    .pipe(merge_categories, ['bancos_em_couro', 'bancos_de_couro', 'bancos_de_camurca', 'bancos_eletricos_em_couro'], 'BANCO_DE_COURO')
    .pipe(merge_categories, ['banco_motorista_com_regulagem_de_altura', 'banco_com_regulagem_lombar', 
                             'banco_do_motorista_com_ajuste_de_altura', 'banco_motorista_c/_reg._altura',
                             'banco_do_motorista_ajuste_de_altura', 'banco_do_motorista_com_ajuste_eletrico',
                             'banco_eletrico', 'banco_com_regulagem_de_altura', 'banco_regulagem_de_altura', 'bancos_com_memoria_de_regulagem'], 'REGULAGEM_BANCO_MOTORISTA')
    .pipe(merge_categories, ['multimidia', 'kit_multimidia', 'multimidia/_camera_de_re', 'touch_screen', 'som_multimidia',
                             'espelhamento_de_tela_celular', 'radio_com_bluetooth', 'midia_bluetooth/usb','central_multim&',
                             'central_multimidia_bluetooth', 'dvd_player', 'espelhamento_de_tela_do_celular', 'kit_multimidia_(gps_dvd)',
                             'media_nav', 'som_touchscreen,_app-connect', 'media_evolution_com_android_auto®_e_apple_carplay®', 'dvd',
                             'android_auto', 'bluetooth', 'kit_multimidia_(gps,dvd)', 'espelhamento_de_celular', 'apple_carplay', 'gps',
                             'espelhamento_tela_celular', 'mp5_player_+_usb_/_tela', 'espelhamento_da_tela_do_celular', 'central_multimidia'], 'CENTRAL_MULTIMIDA')
    .pipe(merge_categories, ['freios_abs', 'freio_abs', 'abs', 'freios_de_disco_dianteiros', 'freio_a_disco_nas_2_rodas',
                             'abs+ebd+pba', 'freio_a_disco_nas_4_rodas', 'freio_a_disco', 'abs+ebd'], 'ABS')
    .pipe(merge_categories, ['parachoque_na_cor_do_veiculo', 'para-choques_na_cor_do_veiculo', 'retrovisores_na_cor_do_veiculo',
                             'parachoques_na_cor', 'retrovisores_na_cor', 'retrovisor_na_cor_do_veiculo', 'kit_na_cor',
                             'para-choque_na_cor_do_veiculo', 'para-choque_pintado_na_cor_do_veiculo', 'parachoque_na_cor_do_ve&',
                             'parachoque_na_cor'], 'PARACHOQUE_RETROVISOR_COLORIDO')
    .pipe(merge_categories, ['cd_player_original', 'cd_player', 'audio_-_sistema_de_som', 'som', 'radio_ou_am/fm', 'radio', 
                             'som_pen_drive', 'radio_bluetooth', 'som_pendrive', 'alto_falante', 'som_original', 'mp3',
                             'radio_com_entrada_usb', 'radio_e_toca_fitas', 'cd_mp3_player', 'radio_fm/am', 'toca_fitas_originais',
                             'conexao_bluetooth', 'radio_usb', 'cd_player_mp3', 'som_usb', 'cd_player_com_mp3', 'radio_e_mp3',
                             'sistema_de_som_harman/kardon', 'cd_e_mp3_player', 'audio_(sistema_de_som)', 'toca_fitas', 'mp3_player',
                             'o_bluetooth', 'am/fm', 'radio_com_pen_drive', 'radio_am/fm', 'radio_e_cd_player', 'entrada_auxiliar',
                             'usb', 'entrada_usb', 'cartao_sd'], 'RADIO')
    .pipe(merge_categories, ['controle_de_som_no_volante', 'comando_de_audio_e_telefone_no_volante', 'volante_ajuste_profundidade',
                             'volante_com_regulagem_de_altura_e_profundidade', 'comandos_no_volante', 'controle_som_no_volante',
                             'volante_multifuncional', 'controle_no_volante', 'volante_com_multi_funcoes', 'cd_controle_no_volante',
                             'volante_com_multifuncao'], 'CONTROLES_NO_VOLANTE')
    .pipe(merge_categories, ['controle_de_tracao', 'controle_de_estabilidade', 'controle_de_tra&', 
                             'controle_eletronico_de_estabilidade_(esp)_com_auxilio_de_partida'], 'CONTROLE_DE_ESTABILIDADE')
    .pipe(merge_categories, ['start-stop', 'sistema_start/stop', 'sistema_stopstart', 'stop_&_start', 'start_stop', 
                             'partida_start_stop', 'botao_start/stop', 'botao_start_stop', 'partida_por_controle_remoto',
                             'acionamento_remoto_do_motor', 'sistema_keyless', 'botao_de_partida', 'sistema_start-stop',
                             'keyless_entry_system', 's.stop', 'partida_no_botao', 'partida_eletrica', 'start_/_stop_engine'], 'START_STOP')
    .pipe(merge_categories, ['vidros_eletricos_dianteiros_e_traseiros', 'vidros_eletricos_nas_4_portas', 'vidro_eletrico_nas_4p', 
                             'vidros_eletricos_traseiro', 'vidros_eletricos_dianteiro', 'vidros_eletricos', 'vidro_eletrico', 'alarme_acionamento_vidros',
                             'vidro_eletrico_nas_4_portas', 'vidro_eletrico_dianteiro', 'vidros_eletrico_nas_4_portas', 'modulo_de_subida_de_vidros',
                             'vidros_eletricos_traseiros', 'alarme_com_acionamento_dos_vidros', 'vidros_el&', 'vidros_eletricos_04_portas',
                             'vidros_eletricos_nas_4p', 'trio_eletrico', 'vidro_eletrico_traseiro', 'vidros_traseiros_eletricos',
                             'vidros_dianteiros_eletricos', 'vidros_eletricos_dianteiros', 'fechamento_automatico_dos_vidros'], 'VIDRO_ELETRICO')
    .pipe(merge_categories, ['espelho_retrovisor_retratil', 'retrovisor_com_rebatimento_eletrico', 'retrovisores_eletricos_com_rebatimento',
                             'retrovisor_eletrico', 'espelhos_externos_com_inclinacao_automatica_em_marcha_re', 'retrovisor_eletrico',
                             'espelhos_el&', 'retrovisores_el&', 'retrovisor_retratil', 'ajuste_eletrico_dos_retrovisores', 'espelhos_eletricos',
                             'controle_eletrico_para_os_retrovisores', 'retrovisores_eletricos', 'retrovisor_eletrico', 'espelho_retrovisor_eletrocromico'], 'RETROVISOR_ELETRICO')
    .pipe(merge_categories, ['farol_de_milha_/_neblina', 'farois_de_neblina_dianteiros', 'farol_de_neblina', 'farois_de_neblina', 'farois_de_milha', 
                             'luz_de_neblina_traseiro', 'is_de_neblina', 'farol_de_milha'], 'FAROL_DE_MILHA')
    .pipe(merge_categories, ['farol_de_xenon', 'farol_xenonio', 'farol_de_xenon_/_led', 'farois_em_led', 'farois_de_led', 'farol_mascara_negra',
                             'farois_full_led', 'pisca_em_led', 'farol_xenon', 'farois_de_xenon', 'farois_diurnos_de_led', 'luz_diurna_de_led', 
                             'luzes_led_diurno', 'farol_de_rodagem_diurna', 'farol_de_xenonio', 'farois_xenon_bi-direcionais', 'farol_de_led',
                             'farois_mascara_negra', 'is_em_led', 'luz_conducao_diurna'], 'FAROL_XENON_LED')
    .pipe(merge_categories, ['unico_dono', '2o_dono', 'segunda_dona', '2_dono', '2._dono', '3._dono', 'garantia_de_fabrica',
                             'unica_dona', '2.dono', 'carro_unico_dono_pneus_ainda', 'na_garantia_de_fabrica', 'na_garantia_de_fabrica_ainda', '0_basico', '0_basico_flex',
                             '2_anos_de_garantia', 'zero_km', 'segundo_dono', 'carro_impecavel_unico_dono', 'na_garantia', '0_km', 'realmente_novo'], 'FEW_OWNERS')
    .pipe(merge_categories, ['banco_traseiro_bipartido_e_rebativel_para_ampliacao_do_porta-malas', 'banco_traseiro_bipartido_e_rebativel',
                             'banco_traseiro_bipartido', 'banco_tras._com_encosto_bipartido', 'banco_traseiro_bipartido_e_rebativel',
                             'banco_bipartido', 'banco_bi-partido'], 'BANCO_TRASEIRO_BIPARTIDO')
    .pipe(merge_categories, ['sistema_de_frenagem_auto?nomo', 'freio_de_estacionamento_com_acionamento_eletronico', 'tracao_asr',
                             'auto_hold', 'ass._de_frenagem', 'frenagem_automatica_de_emergencia'], 'FRENAGEM_AUTONOMA')
    .pipe(merge_categories, ['alerta_de_mudanca_de_faixa', 'alerta_de_limite_de_velocidade', 'indicador_de_fadiga',
                             'alerta_trafego_cruzado_traseiro_e_dianteiro', 'toyota_safety_sense_(tss)', 'sistema_drive_mode',
                             'direcao_assistida', 'controle_aut._de_velocidade', 'controle_de_velocidade', 'controle_automatico_velocidade', 'alarme_velocidade',
                             'transito_em_tempo_real', 'leitor_de_faixa', 'piloto_automatico', 'alerta_ponto_cego', 'controle_automatico_de_velocid',
                             'alerta_de_ponto_cego', 'assistente_de_permanencia_em_faixa', 'sensor_de_mudanca_de_pista', 'sensor_de_fadiga',
                             'controle_de_velocidade_adaptativo', 'piloto_automatico_adaptativo', 'ass._de_faixa'], 'DRIVER_ASSISTANCE')
    .pipe(merge_categories, ['turbo_intecooler', 'turbo', 'turbo_forjado', 'turbo_intercooler'], 'TURBO')
    .pipe(merge_categories, ['sensor_de_estacionamento_dianteiro', 'sensor_aproximacao', 'assist_de_estac_semi-aut', 'sensor_de_estacionamento',
                            'sens_de_estac_diant/traz', 'mera_de_r&', 'alarme_sonoro_de_re', 'alerta_colisao', 'sens_de_estac_dianteiro', 'camera_360_graus', 
                            'sensor/camera_de_re',  'sensor_de_re', 'sensores_de_estac_dianteiro', 'sensor_de_est._dianteiro', 'sensor_dianteiro_de_estacionamento',
                            'sensor_estacionamento_dianteiro', 'estaciona_sozinho', 'park_assist', 'camera_de_re', 'sensor_de_estacionamento_traseiro', 'camera_360°', 
                            'alerta_de_colisao_frontal', 'multimidia/gps/camera_de_re', 'alerta_de_colisao', 'camera_de_re_360*', 
                            'assist._de_estacionamento_(park_assist)'], 'ESTACIONAMENTO')
    .pipe(merge_categories, ['teto_solar_panoramico', 'teto_solar', 'teto_solar_duplo'], 'TETO_SOLAR')
    .drop(['whatsapp_19_3025-0323', '#243', '#231', '7_lugares!!!_impecavel!!!!!!', 'otimo_estado_de_conservacao', 'oleo_e_filtros_trocados',
           'licenciado_2022', 'licenciado_ate_agosto/2025', 'revisado_concessionaria', 'revisoes_feitas_na_concessionaria', 'breake_light',
           'cautelar_aprovada', 'nota_fiscal', 'veiculo_nao_fumante', 'correia_dentada_revisada', 'ipva_2024_total_pago', 'pisca_no_retrovisor',
           'ipva_pago', 'sem_capota_de_fibra_r$_72.000', 'versao_top_de_linha_-_run', 'estepe', 'baixa_km', 'baixa_kilometragem', 'aceito_troca',
           'molas_preparadas', 'impecavel', 'motor_0km', 'corro_impecavel', 'documento_2024_ok', 'licenciado', 'console_central', 'veiculo_alienado',
           'volante_esportivo', 'cabine_extendida', 'manual_do_carro', 'manual_e_chave_co', 'cabine_dupla',
           'bancos_dianteiros_reclinaveis', 'alto_falantes_dianteiros', 'relogio_digital', 'todas_revisoes_feitas_na_css', 'frisos_laterais',
           'modelo_g3', 'suspensao_de_molas', 'sico', '#244', 'engate_de_carretinha', 'pneus_novos', '2_pneus_dianteiros_novos', 'encosto_cab._traseiro',
           'display_com_relogio_digital', 'sensor_de_iluminacao_(crepuscular)', 'bloqueador', 'cambio_tiptronic', 'direcao_mecanica', 'code',
           'gaveta_porta-objetos_sob_o_banco_do_motorista', 'licenciamento_pago','sensor_de_farol', 'alarme_de_luzes_acesas',
           'manual_do_propriet&', 'garantia_de_3_meses', 'quebra-mato', 'aerofolio_traseiro', 'chamada_emergencia', 'nio',
           'de_uso_pessoal', 'sem_gravame', 'alerta_de_manutencao_programada', 'insulfilm_carbono', 'r&', 'rack_de_teto', 
           'para-sol_com_espelho_para_o_passageiro', 'revisado_em_concessionaria', 'distribuicao_eletronica_de_frenagem_(ebd)', 'aerofolio',
           'monitoramento_da_pressao_dos_pneus_(tpms)', 'apoio_de_braco_lateral', 'retrovisor_fotocr&', 'chave_presencial', 
           'acendimento_automatico_dos_farois', 'antena_eletrica', 'maanual_chave_reserva', 'vidros_verdes', 'cambio_automatico',
           'bancos_com_isofix', 'encosto_de_cabe&','trocador_de_calor_novo', 'temporizador_do_limpador_do_para-brisa', 'pintura_met&', 
           'assistente_partida_em_rampa', 'acessibilidade','transmissao_tiptronic', 'destaque', 'trava_eletrica_de_porta_malas', 'parabrisa_degrade', 
           'indicador_de_temperatura_externa', 'repetidores_laterais_das_luzes_de_direcao', 
           'protetor_para_motor_e_carter', 'apoio_de_braco', 'regulagem_eletrica_dos_farois', 'limp_de_para-brisa_automatico', 
           'abertura_interna_do_porta-malas', 'vidros_unique', 'acendedor_de_cigarros', 'desembacador_traseiro', 'my_link', 'volante_c/_ajuste_de_profund.', 
           'encosto_de_cabeca_traseiro', 'motohome', 'janela_corredica', 'chave_codificada_(code)', 'kit_gnv_-_gas', 'tricos_com_rebatimento', 
           'monitoramento_pressao_dos_pneu', 'comfortline', 'freio_de_mao_eletrico', 'espelho_cortesia_para_passageiro_e_motorista', 'tricos', 'o_el&', 
           'suporte_para_celular', 'farois_de_neblina_traseiros', 'carregador_wireless', 'trava_carneiro', 'engate_para_reboque', 'apoio_de_braco_para_motorista', 
           'trica', 'pneu_runflat', 'frisos_laterais_na_cor_do_veiculo', 'manual__e_chave_reserva', 'pedal_shift', 'isofix_infantil',
           'faro', 'conta-giros', 'basico', 'cinto_de_seguranca', 'apoio_de_braco_central_deslizante', 'protetor_de_ca&', 'manual_do_usuario', 'realmente_conservado', 
           'licenciado_ate_setembro/2025', 'kit_visibilidade', 'insufilm', 'motor_novo', 'completo', 'apoio_de_bra&', 'lona_maritima', 'abertura_interna_porta_mala', 
           'mico', 'stage_2', 'com_ao_veic_atraves_de_app', 'revisoes_em_concessionaria', 'calha_de_chuva', 'aviso_sonoro_de_farois_acesos', 'paddle_shift', 'disqueteira', 
           'midia_nave', 'banco_reclinavel', 'tracao_4x2', 'volante_escamoteavel_eletrico', 'freio_de_estacionamento_eletronico', 'estribos_laterais', 'chave_canivete', 'kit_gas(gnv)',
           'rodas_aro_17', 'relogio_no_painel', 'dio', 'manual/chave_reserva', 'alto_falantes_traseiros', '..', 'volante_com_regulagem_de_altura', 'luz_de_cortesia_no_teto',
           'descanso_de_pe', 'garantia_de_motor_e_cambio_de', 'desembacador', 'protetor_de_cacamba', 'blindado', 'ft450', 'banco_traseiro_retratil', 'ajuste_eletrico',
           'abertura_interna_do_porta_malas', 'abertura_porta_malas_por_controle_remoto', 'cambio_borboleta', 'direcao_multifuncional', 'painel_jacaranda', 'amba',
           'kit_ferramentas', 'volante_em_couro', 'retrovisores_com_seta', 'acendedor_de_cigarro', 'cambio_semi-automatico', 'tricos_dianteiro', 'baixo_km',
           'polimento', 'estribo_lateral', '6_marchas', 'basico_de_serie', 'abertura_interna_porta_malas', 'revisado_na_concessionaria', 'carreg_de_celular_por_inducao', 
           'volante_c/_ajuste_profundidade', 'backlight', 'partida_remota', 'destravamento_remoto_do_porta_malas', 'baixa_quilometragem', 'manual_+_chave_copia', 
           'farois_c/_acendimento_aut', 'dia', 'amp', 'pneus_hankook_235/60', 'porta-luvas_climatizado', 'bancos_aquecidos', 'tricas', 'rodas_aro_20', 'tampa_traseira_automatizada', 
           'porta_oculos', 'b&', 'lavador_traseiro', 'bagageiro_no_teto', 'cambio_automatico_com_active_select', 'assistente_de_partida_em_rampa', 'ador_traseiro', 
           'assentos_reclinaveis', 'rarissimo_modelo', 'calhas_de_chuva', 'carregador_de_celular_por_inducao', 'o_hidr&', 'bancos_dianteiros_com_aquecimento',
           'carregador_por_inducao', 'computador_de_bordo', 'cambio_automatizado', 'bancos_aqu', 'o', 'retrovisor_fotocromico', 'passagem_por_leilao',
           'blindado_axxis', 'assistencia_recuperacao_veicular', 'injecao_eletronica', 'ebd', 'sistema_isofix_para_cadeira_infantil', 'cambio_semi_automatico',
           'abertura_interna_da_tampa_de_combustivel', 'laudo_cautelar', 'insulfilme', 'chave_com_sensor_de_presenca', 'regulagem_de_volante', 'travamento_portas', 
           'o_4x2', 'esguicho_de_farol', 'sensor_de_chuva', 'tracao_4x4', 'painel_digital', 'roteador_wi-fi', 'apoios_de_cabeca_traseiros', 'sensor_de_luz', 
           'manual_do_proprietario', 'volante_c/_regulagem_altura', 'kit_gas', 'assistente_em_rampa/_descida', 'rio', 'triangulo', 'chave_de_roda', '#226',
           'ragtop', '#225', 'para-sol_com_espelho_para_o_motorista', 'luz_de_leitura',  'macaneta_na_cor_do_veiculo', 'com_injecao_eletronica', 'alarme_inteligente', 
           '7_lugares', 'banco_eletrico_para_passageiro', 'o_4x4', 'tima', 'spoiler', 'encosto_passageiro', 'farois_automaticos', 'retrovisor_com_pisca', 'manual', 
           'padle_shift', 'licenciado_ate_nov/2025', 'luz_de_neblina', 'break_light', 'farois_c/_acend_automatico', 'licenciado_ate_outubro/2025', '4x2',
           'bancos_dianteiros_c/_aquec.', 'protetor_de_carter', 'confort_line', 'tricos_traseiro', 'maravilhosa', 'carregador_de_celular_sem_fio', 'estribo', 
           '25.000,00_em_acessorios.', 'assistente_de_rampa', 'limitador_de_velocidade', 'ulica',  'sistema_de_bloqueio_de_ignicao', 'serie_especial',
           'assoalho_em_carpete', '1_ano_de_garantia', 'toda_revisada', 'documentacao_ok', 'comando_interno_na_tampa_do_tanque', 'a_traseiro', 'spoilers', 
           'ipva_quitado', '3_portas', 'todas_revisoes_na_css', 'engate', 'corrente_de_comando', 'regulagem_altura_do_banco', 'pneus_run_flat', 'protetor_de_motor_e_carter', 
           'cambio_dualogic', 'veiculo_de_nao_fumante', 'bloqueio_de_tracao', 'retrovisores_com_regulagem_interna', 'ipva_2025_e_2026_pago_(isento)', 'bancos_revestidos_em_tecido', 
           'tricos_em_couro', 'revisoes_em_dia', 'espelho_cortesia_para_passageiro', 'chave_reserva', 'suporte_para_estepe', 'limpador_traseiro', '4wd', 'frisos_cromados',
           'de_fabrica_,com_todos_os_selos', 'cintos_de_seg._c/_reg._de_altura', 'rack_no_teto', 'brake_light', 'farois_com_regulagem_de_altura', 'timo_estado_de_conservacao', 
           'laudo_cautelar_aprovado', 'santo_antonio', 'quilometragem_baixa', 'cambio_de_9_marchas', 'limpador_e_desembacador_traseiro', 'setas_no_retrovisor', 
           'inf_do_veiculo_atraves_de_app', '.', 'espelho_de_cortesia_p/_motorista', 'escapamento_esportivo', 'engate_traseiro', 'escape_6x2', 'pia', 'lona_mar&', 
           '2_pneus_traseiros_meia_vida', 'retrovisor_fotocromatico', 'macanetas_na_cor_do_veiculo', 'hx35', 'carregador_do_celular_via_inducao', 
           '#237', 'bancos_dianteiros_com_regulagem_eletrica', 'licenciado_2021', 'chave_de_presenca', '12_volts', 'pneus_semi_novos', 'semiautomatico',
           'conversivel', 'reliquia', 'volante_escamoteavel', 'bancos_esportivos', 'sensor_de_iluminacao', 'aquecimento_dos_bancos', 'motor_ap', 'teto_panoramico', 
           'onstar', 'escapamento', 'cambio_no_volante_(borboleta)', 'harman_kardon_sound_16_falantes', 'chave_copia', 'descanso_de_braco_central', 'revisado', 
           '4_pneus_novos', 'launch_control', 'interior_na_cor_bege', 'on_star', 'porta_malas_automatico', 'watz_19-9-7412-8555', 'abertura_int._da_tampa_do_comb.', 
           'banco_traseiro_com_porta-objetos', 'aceita_troca', 'farois_c/_reg_de_altura', 'freio_de_mao_eletronico', 'macaco', 'aviso_sonoro_dos_farois_acesos', 'friso_lateral', 
           'culo', '4x4', 'assistencia_na_recuperacao_veicular', 'cintos_c/reg._altura', 'desemba&', 'tracao_fwd', 'nao_aceita_troca', 'sensor_farol', 'kit_diplomata', 
           'teto_alto', 'antena_no_teto', '#227', 'chave_copiareserva', 'bancos_com_ajuste_eletrico', 'faixas_laterais', 'capota_mar&', 'banco_motorista_com_ajuste_de_altura', 
           'banco_reccaro', 'ipva_pago_e_licenciado', 'maravilhoso', 'distribuicao_eletr_de_frenagem', 'bancos_el&', 'periciado', 'c&', 'quebra_mato', 
           'comando_interno_de_abertura_do_porta_malas', 'cambio_automatico_tiptronic', 'estepe_nunca_usado', 'banco_motorista_com_regulagem_eletrica', 'cambio_cvt', 
           'porta_copos', 'espelho_retrovisor_interno_dia/noite', 'freio_estacionamento_eletrico', 'porta-copos_no_console_central', 'insul_film', 'placa_mercosul', 
           'possui_painel_de_instrumentos', 'monitoramento_da_pressao_dos_pneus', 'assist._de_partida_em_rampa', 'tra&', 'assistente_partida_em_subida', 'tico', 
           'parachoque_personalizado', 'banco_do_motorista_com_regulagem_de_altura', 'porta-malas_eletrico', 'banco_do_motorista_eletrico', 'santo_ant&', 
           'controle_interno_do_espelho', 'tip_tronic', 'para-brisa_degrade', 'coluna_de_direcao_com_regulagem_de_altura', 'aquecimento_bancos_dianteiros', 'espelho_personalizado', 
           'overlap', 'outros', 'frisos_e_soleiras', 'todo_revisado', 'sensor_de_luminosidade', 'conex&', 'ca-dualogic', 'pintura_metalica', 'porta-copos', 
           'distribuicao_eletronica_de_frenagem', 'calhas_de_acrilico', 'importado', 'capota_de_lona', 'regulagem_de_altura_farois', 'controle_automatico_de_descida', 
           'console_de_teto', '2_pneus_novos', 'capota_maritima', 'direcao_escamoteavel', 'revisoes_na_concessionaria', 'coluna_de_direcao_ajustavel',
           'refrigeracao_liquida', 'bancos_recaro', 'pelicula_solar_(insulfilm)', 'antena_interna', 'lica', 'higienizacao_de_bancos', 'licenciado_ate_novembro/2025', 
           'manual_proprietario', 'carro_impecavel', 'tracao_integral_4x4', 'interface', 'partida_na_chave', 'cambio_manual', 'painel_colorido', 'farois_auxiliares', 
           'volante_ajuste_altura', 'bancos_eletricos', 'isofix_p_fixar_cadeirinha', 'pelicula_anti_-_vandalismo', 'calotas', 'raridade', 'isofix', 'farois_duplos', 
           'capota_de_fibra', 'head-up_display', 'detalhes_em_cromo_(motor)', 'manual_+_chave_co', 'piloto_autom&', 'turbina_instalada', 'chave_com_telecomando', 
           'servico_de_concierge', 'banco_com_regulagem_eletrica', 'cintos_de_seguraca_com_regulagem_de_altura', 'porta_malas_eletrico', 'porta_copos_traseiro_no_console',
           'laudo_cautelar_ok', 'banco_traseiro_rebativel', 'pneus_bons', 'trava_central_portas', 'assist._de_rampa', 'cambio_no_volante', 'teto_solar_eletrico', 
           'sistema_isofix', 'controle_tracao_/_estabilidade', 'farois_com_refletores_duplos', 'bagageiro', '1', 'apoio_de_braco_central', 'controle_automatico_de_velocidade', 
           'porta_luvas_climatizado', 'chave_com_codigo', 'estepe_sem_uso', 'automatico', 'pelicula_solar', 'far&', '&', 'placa_preta', 'abertura_int_tampa_combustivel', 
           'revisado_no_agenda_do_carro', 'manual_e_chave_reserva', 'seta_no_retrovisor', '#233', 'cabine_estendida', 'motor_e_cambio_ok!', 'farois_com_regulagem_interna'], axis=1)
    .assign(KIT_DIGNIDADE = lambda x: np.where((x['AR'] == 1) &
                                               (x['DIRECAO'] == 1) &
                                               (x['VIDRO_ELETRICO'] == 1) &
                                               (x['TRAVAS'] == 1) &
                                               (x['CENTRAL_MULTIMIDA'] == 1) &
                                               (x['AIRBAG'] == 1) &
                                               (x['ABS'] == 1), 1, 0))
)

In [ ]:
car_features#.to_csv('data/car_features.csv', index=False)